<div style="text-align:center;color:Blue">
    <h2> Large Language Model (LLM) sentiment analysis of financial news using Alpaca, BERT and ChatGPT</h2>
</div>

#### An experiment  with the fascinating potential of large language models to efficiently classify short news summaries and headlines into 'positive', 'neutral' and 'negative' sentiments. Here we use Alpaca, BERT and ChatGPT to:
1. import news headings and summaries for specific stock symbols
2. use the language (NLP) model Bidirectional Encoder Representations from Transformers (BERT) to tokenize and classify the news data into sentiments 
3. classify the same news data using OpenAI's gpt-3.5 to do the same 

#### Step 0 - Install the necessary libraries - uncomment and run the following cells if they are not already installed. This is followed by importing the necessary libraries

In [1]:
# !pip -q install alpaca-trade-api alpaca-py transformers openai tiktoken

In [2]:
# Langchain sometimes has issues with dependencies and it is recommended to do the following:
# !pip -q install langchain --upgrade

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from enum import Enum
from pydantic import BaseModel, Field
from alpaca_trade_api import REST, Stream
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModelForSeq2SeqLM
from datetime import date, timedelta
import pandas as pd
import numpy as np
import openai
import os

2023-07-13 16:37:31.605201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<div style="text-align:center;color:red">
    <h2>1 - Import keys saved in your environment</h2>
</div>

#### Most APIs provide a security option to ensure that you can store your authentication details in environment variables. This enables you:
1. to authenticate your logins and code tracking in code depos such as GitHub
2. in addition to #1, to control your privileges such as OpenAI's tokens
3. to protect yourself from inadvertently exposing your secret IDs and keys in any environment such as live trading platforms such as Alpaca
#### For best practices read:
1. OpenAI's advice: https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
1. Alapaca: https://medium.com/software-engineering-learnings/algorithmic-trading-with-alpaca-and-python-c81bad480053
3. General: https://www.twilio.com/blog/how-to-set-environment-variables-html

In [4]:
openai.api_key = os.environ["OPENAI_API_KEY"]
API_KEY = os.getenv("APCA_API_KEY_ID")
API_SECRET = os.getenv("APCA_API_KEY_SECRET")
print('keys imported')

keys imported


<div style="text-align:center;color:red">
    <h2> 2 - Setup your llm model and construct a pydantic base model</h2>
</div>

##### Pydantic defines objects via models (classes which inherit from pydantic.BaseModel).
More info here: https://docs.pydantic.dev/latest/usage/models/ which states:
- These models are similar to Python's dataclasses with some differences that streamline certain workflows related to validation, serialization, and JSON schema generation. Untrusted data can be passed to a model and, after parsing and validation, Pydantic guarantees that the fields of the resultant model instance will conform to the field types defined on the model.

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

class Tags(BaseModel):
    sentiment: str = Field(..., enum=["positive", "neutral", "negative"])

chain = create_tagging_chain_pydantic(Tags, llm)

<div style="text-align:center;color:red">
    <h2> 3 - Prepare sentiment analysis models and pipeline</h2>
</div>

#### Approach and cautionary note:
- we will use a fine-tuned (trained for financial news) Hugging Face model (BERT) to analyze the article's headline and summary sentiment
- initial model downloads might take some time

In [6]:
sent_tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
sent_model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
sent_nlp = pipeline("sentiment-analysis", model=sent_model, tokenizer=sent_tokenizer)

#### Next, create an alpaca client, choose stock tickers and decide how many days of news to scrape

In [7]:
rest_client = REST(API_KEY, API_SECRET)
tickers = ["GOOG","AAPL","YAH",'MSFT','AAPL','CNN','KHC']
number_of_days = 10
start_date = date.today() - timedelta(number_of_days)
end_date = date.today()

<div style="text-align:center;color:red">
    <h2> 4 - Create a functions to classify individual news and calculate an average sentiment per ticker and to process all tickers</h2>
</div>

#### Important considerations:
- more recent news might be more relevant
- the sentiment confidence also gives us a clue how certain the algorithm is about its classification

#### Approach:
- weigh recent news more heavily (straightforward linear increase, going from old to new - although there can be many variations of this approach such as inverted/hyperbolic, linear-with-noise, etc. approaches)
- use sentiment confidence to adjust our weights. i.e. multiply recency score with the score
- the function ```sentiment_to_weighed``` takes care of the weighing
- the function ```sentiment_analysis``` takes in a list and tickers and returns a weighed sentiment per ticker
- since OpenAI's tocken allotments deplete quickly, a few lines in  ```sentiment_analysis``` are commented out and a flag is given. Uncomment them if you have enough tokens left (after changing ```do_llm``` flag to "1". Note that llm sentiment classification is slow.

In [8]:
def sentiment_to_weighed(sent_labels, sent_scores):
    if len(sent_labels):
        len_news = len(sent_labels)
        sentiment_values = [0.9 if x == 'positive' else 0.5 if x == 'neutral' else 0.3 
                                 for x in sent_labels.values]*sent_scores
        sent_weights = np.array(list(reversed(np.arange(0.1, (len_news)/len_news+0.1, 1/len_news))))
        sent_scored_weights = sentiment_values*sent_weights 
        mean_sent_scored = np.average(sentiment_values, weights = sent_scored_weights)
        sent_weighed = ['positive' if mean_sent_scored >= 0.65 else 'neutral' 
                        if (mean_sent_scored >= 0.3 and mean_sent_scored <0.65) 
                        else 'negative'][0]
    else:
        print('no news')
        sent_weighed = 'neutral'
    return sent_weighed 

In [9]:
def sentiment_analysis(news_sents, tickers, do_llm):
    news_sents_columns = ['ticker','sentiment']
    for t in tickers:
        news_sentiments = pd.DataFrame()
        news = rest_client.get_news(t, start_date, end_date)
        news1 = pd.DataFrame({
            'summary': [x.summary for x in news],
            'headline': [x.headline for x in news],
    #         uncomment the following to verify that news are sorted most recent first
    #         created_at': [x.created_at for x in news], 
        })
        news_sentiments['headline&summary'] = news1.summary + news1.headline
        sentiment = [sent_nlp([x])[0] for x in news_sentiments['headline&summary']]
        
        if do_llm:
            news_sents_columns = ['ticker','sentiment', 'llm_sentiment']
            llm_sentiment = [chain.run(x).sentiment for x in positions['headline&summary']]
            news_sentiments['llm_label'] = [llm_sentiment[x] for x in range(0, len(llm_sentiment))]
            # to save time, we use a score of 1 for llms - although this can be done exactly as BERT
            llm_scores = [1]*len(llm_sentiment)
            sent_llm_wighed = sentiment_to_weighed(news_sentiments['llm_label'], llm_scores)
            
        news_sentiments['label'] = [sentiment[x]['label'] for x in range(0, len(sentiment))]
        news_sentiments['score'] = [sentiment[x]['score'] for x in range(0, len(sentiment))]
        sent_wighed = sentiment_to_weighed(news_sentiments['label'], news_sentiments['score'])
        news_sentiments['ticker'] = t
        
        # peek at the results - to save time and visual space, show only the first ticker resuls
        if t == tickers[0]:
            print(news_sentiments)
        #uncomment the following line to see dataframes for each ticker and news article
#         print(news_sentiments)
        
        if do_llm:
            news_sents.append([t, sent_wighed, sent_llm_wighed])
        else:
            news_sents.append([t, sent_wighed])
            
    news_sents = pd.DataFrame(news_sents)
    news_sents.columns = news_sents_columns
    return news_sents

In [10]:
news_sents = []
do_llm = 0
sentiments_df = sentiment_analysis(news_sents, tickers, do_llm)

                                    headline&summary     label     score  \
0   A new player has joined the AI chatbot race, ...  positive  0.950089   
1  U.S. Sen. Bernie Sanders has made waves over t...   neutral  0.999205   
2  Google Play Changes Policy On Tokenized Digita...   neutral  0.999427   
3  'The Roku Channel Is Now Streaming On Google T...   neutral  0.990214   
4  Wall Street JournalDisney Considers Different ...   neutral  0.999546   
5  Alphabet Inc (NASDAQ: GOOG) (NASDAQ: GOOGL) Go...   neutral  0.997340   
6  George Hotz discusses the unique decision-maki...   neutral  0.999708   
7  Elon Musk&#39;s comments on chatGPT&#39;s hall...  positive  0.700688   
8  Each trading day features hundreds of headline...   neutral  0.999608   
9  To gain an edge, this is what you need to know...   neutral  0.961198   

  ticker  
0   GOOG  
1   GOOG  
2   GOOG  
3   GOOG  
4   GOOG  
5   GOOG  
6   GOOG  
7   GOOG  
8   GOOG  
9   GOOG  
no news
no news


In [11]:
sentiments_df

,ticker,sentiment
0,GOOG,neutral
1,AAPL,positive
2,YAH,neutral
3,MSFT,positive
4,AAPL,positive
5,CNN,neutral
6,KHC,neutral


### Article Summarization
After importing financial news for various tickers using Alpaca, a combination of Google's BERT and OpenAI's GPT was used to provide average sentiments. Thanks to the greate architects and trainers of the models, implementing the Hugging Face Transformers library is relatively easy. OpenAI's GPT llm model:
1. was slow 
2. it was not specifically trained for financial news
3. can not be run for large tokes due to specific quotas
These facts made it impossible to do a direct comparisons at this time but the given functions can be used to do so in a different approach/time.
